<a href="https://colab.research.google.com/github/ManuelaCastano/CuadernosyEjercicios/blob/main/C%C3%B3digoDashboard_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CÓDIGO DEL DASHBOARD

In [ ]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

In [ ]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile #librerias descarga Youtube y manejo de audios en python

  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached websockets-13.0.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cach

In [ ]:
!pip install pydub #instalamos pydub para manipular el audio

In [ ]:
"""%%writefile Parcial2.py

#GRAFICAS CON ST.PYPLOT

import streamlit as st  # permite crear aplicaciones web interactivas directamente desde Python
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib

from scipy.fft import fft, fftfreq
from IPython.display import Audio
from pydub import AudioSegment
import io

# Intentar importar yt_dlp y manejar la falta de instalación
try:
    import yt_dlp
except ImportError:
    st.error("El módulo 'yt_dlp' no está instalado. Por favor, instálalo usando 'pip install yt-dlp'.")
    st.stop()  # Detener la ejecución si no se puede importar yt_dlp

# Descargar el audio de YouTube y convertir a WAV
def descargar_audio_youtube(url):
    ydl_opts = {
        'format': 'bestaudio[ext=m4a]',
        'outtmpl': 'audio.m4a'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # Convertir a WAV
    audio = AudioSegment.from_file("audio.m4a", format="m4a")
    audio.export("audio.wav", format="wav")
    os.remove("audio.m4a")  # Limpiar el archivo m4a temporal

# Cargar el archivo de audio
def cargar_audio(filename):
    try:
        data, sample_rate = sf.read(filename)
        return sample_rate, data
    except FileNotFoundError:
        st.error(f"No se encontró el archivo {filename}. Asegúrate de que el archivo exista y esté en la ubicación correcta.")
        return None, None

# Convertir audio estéreo a mono
def convertir_a_mono(data):
    if len(data.shape) > 1:  # Si es estéreo
        return np.mean(data, axis=1)
    return data

# Graficar la señal en el tiempo usando Streamlit
def graficar_senal_tiempo_st(t, signal, title):
    st.line_chart(pd.DataFrame({'Tiempo': t, 'Amplitud': signal}))

# Graficar la transformada de Fourier usando Streamlit
def graficar_fft_st(t, signal, title):
    N = len(signal)
    T = t[1] - t[0]
    yf = fft(signal)
    xf = fftfreq(N, T)[:N // 2]

    st.line_chart(pd.DataFrame({'Frecuencia [Hz]': xf, 'Magnitud': np.abs(yf[0:N // 2])}))

# Función principal de la app Streamlit
def main():
    st.title('Simulación de Modulación por Amplitud (AM)')

    url = st.text_input("Introduce el enlace del video de YouTube")

    if url:
        st.write("Descargando audio de YouTube...")
        descargar_audio_youtube(url)

        # Cargar el archivo de audio convertido
        sample_rate, data = cargar_audio("audio.wav")
        if data is None:
            return  # Salir si el archivo no se encuentra

        # Convertir a mono si es estéreo
        data_segment = convertir_a_mono(data)

        # Leer un fragmento de 5 segundos del archivo de audio
        ti = 20  # tiempo de inicio en segundos
        tf = 25   # tiempo final en segundos
        data_segment = data_segment[int(ti * sample_rate):int(tf * sample_rate)]

        # Tiempo de la señal
        t = np.arange(len(data_segment)) / sample_rate

        # Mostrar la señal de audio original
        st.subheader('Señal de Audio Original')
        graficar_senal_tiempo_st(t, data_segment, 'Señal de Audio en el Tiempo')

        # Transformada de Fourier de la señal de audio
        st.subheader('Transformada de Fourier de la Señal de Audio')
        graficar_fft_st(t, data_segment, 'Espectro de la Señal de Audio')

        # Parámetros de la portadora
        Fc = st.number_input('Frecuencia de la portadora (Hz)', value=15000)
        Im = st.slider('Índice de Modulación', 0.0, 1.0, 0.5)

        # Cálculo de la señal portadora y modulada
        Ac = np.max(np.abs(data_segment)) / Im
        c = Ac * np.sin(2 * np.pi * Fc * t)
        y = (1 + Im * data_segment / Ac) * c

        # Mostrar la señal portadora
        st.subheader('Señal Portadora')
        graficar_senal_tiempo_st(t, c, 'Señal Portadora en el Tiempo')

        # Transformada de Fourier de la señal portadora
        st.subheader('Transformada de Fourier de la Señal Portadora')
        graficar_fft_st(t, c, 'Espectro de la Señal Portadora')

        # Mostrar la señal modulada
        st.subheader('Señal Moduladora AM')
        graficar_senal_tiempo_st(t, y, 'Señal Moduladora AM en el Tiempo')

        # Transformada de Fourier de la señal modulada
        st.subheader('Transformada de Fourier de la Señal Modulada')
        graficar_fft_st(t, y, 'Espectro de la Señal Modulada')

        # Reproducción de audio
        st.subheader('Reproducción de Audio')

        # Convertir el segmento de datos a WAV en un buffer de BytesIO
        wav_buffer = io.BytesIO()
        sf.write(wav_buffer, data_segment, sample_rate, format='wav')
        wav_buffer.seek(0)  # Reiniciar el puntero del buffer

        st.audio(wav_buffer, format='audio/wav')

        os.remove("audio.wav")

if __name__ == "__main__":
    main()"""


'%%writefile Parcial2.py\n\nimport streamlit as st  # permite crear aplicaciones web interactivas directamente desde Python\nimport pandas as pd\nimport numpy as np\nimport os\nimport subprocess\nimport soundfile as sf\nfrom sklearn.preprocessing import MinMaxScaler\nfrom sklearn.manifold import TSNE\nfrom pyngrok import ngrok\nimport yt_dlp as youtube_dl\nimport joblib\n\nfrom scipy.fft import fft, fftfreq\nfrom IPython.display import Audio\nfrom pydub import AudioSegment\nimport io\n\n# Intentar importar yt_dlp y manejar la falta de instalación\ntry:\n    import yt_dlp\nexcept ImportError:\n    st.error("El módulo \'yt_dlp\' no está instalado. Por favor, instálalo usando \'pip install yt-dlp\'.")\n    st.stop()  # Detener la ejecución si no se puede importar yt_dlp\n\n# Descargar el audio de YouTube y convertir a WAV\ndef descargar_audio_youtube(url):\n    ydl_opts = {\n        \'format\': \'bestaudio[ext=m4a]\',\n        \'outtmpl\': \'audio.m4a\'\n    }\n\n    with yt_dlp.YoutubeDL

In [ ]:
%%writefile Modulacion.py

import streamlit as st #permite crear aplicaciones web interactivas directamente desde Python
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib

from scipy.fft import fft, fftfreq
from IPython.display import Audio
from pydub import AudioSegment
import io

# Intentar importar yt_dlp y manejar la falta de instalación
try:
    import yt_dlp
except ImportError:
    st.error("El módulo 'yt_dlp' no está instalado. Por favor, instálalo usando 'pip install yt-dlp'.")
    st.stop()  # Detener la ejecución si no se puede importar yt_dlp

# Descargar el audio de YouTube y convertir a WAV
def descargar_audio_youtube(url):
    ydl_opts = {
        'format': 'bestaudio[ext=m4a]',
        'outtmpl': 'audio.m4a'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # Convertir a WAV
    audio = AudioSegment.from_file("audio.m4a", format="m4a")
    audio.export("audio.wav", format="wav")
    os.remove("audio.m4a")  # Limpiar el archivo m4a temporal

# Cargar el archivo de audio
def cargar_audio(filename):
    try:
        data, sample_rate = sf.read(filename)
        return sample_rate, data
    except FileNotFoundError:
        st.error(f"No se encontró el archivo {filename}. Asegúrate de que el archivo exista y esté en la ubicación correcta.")
        return None, None

# Convertir audio estéreo a mono
def convertir_a_mono(data):
    if len(data.shape) > 1:  # Si es estéreo
        return np.mean(data, axis=1)
    return data

# Graficar la señal en el tiempo
def graficar_senal_tiempo(t, signal, title, xlabel, ylabel):
    plt.figure(figsize=(10, 4))
    plt.plot(t, signal)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid()
    st.pyplot()

# Graficar la transformada de Fourier
def graficar_fft(t, signal, title):
    N = len(signal)
    T = t[1] - t[0]
    yf = fft(signal)
    xf = fftfreq(N, T)[:N // 2]

    plt.figure(figsize=(10, 4))
    plt.plot(xf, np.abs(yf[0:N // 2]))
    plt.title(title)
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Magnitud')
    plt.grid()
    st.pyplot()

# Función principal de la app Streamlit
def main():
    st.title('Simulación de Modulación por Amplitud (AM)')

    url = st.text_input("Introduce el enlace del video de YouTube")

    if url:
        st.write("Descargando audio de YouTube...")
        descargar_audio_youtube(url)

        # Cargar el archivo de audio convertido
        sample_rate, data = cargar_audio("audio.wav")
        if data is None:
            return  # Salir si el archivo no se encuentra

        # Convertir a mono si es estéreo
        data_segment = convertir_a_mono(data)

        # Leer un fragmento de 5 segundos del archivo de audio
        ti = 20  # tiempo de inicio en segundos
        tf = 25   # tiempo final en segundos
        data_segment = data_segment[int(ti * sample_rate):int(tf * sample_rate)]

        # Tiempo de la señal
        t = np.arange(len(data_segment)) / sample_rate

        # Mostrar la señal de audio original
        st.subheader('Señal de Audio Original')
        graficar_senal_tiempo(t, data_segment, 'Señal de Audio en el Tiempo', 'Tiempo [s]', 'Amplitud')

        # Transformada de Fourier de la señal de audio
        st.subheader('Transformada de Fourier de la Señal de Audio')
        graficar_fft(t, data_segment, 'Espectro de la Señal de Audio')

        # Parámetros de la portadora
        Fc = st.number_input('Frecuencia de la portadora (Hz)', value=15000)
        Im = st.slider('Índice de Modulación', 0.0, 1.0, 0.5)

        # Cálculo de la señal portadora y modulada
        Ac = np.max(np.abs(data_segment)) / Im
        c = Ac * np.sin(2 * np.pi * Fc * t)
        y = (1 + Im * data_segment / Ac) * c

        # Mostrar la señal portadora
        st.subheader('Señal Portadora')
        graficar_senal_tiempo(t, c, 'Señal Portadora en el Tiempo', 'Tiempo [s]', 'Amplitud')

        # Transformada de Fourier de la señal portadora
        st.subheader('Transformada de Fourier de la Señal Portadora')
        graficar_fft(t, c, 'Espectro de la Señal Portadora')

        # Mostrar la señal modulada
        st.subheader('Señal Moduladora AM')
        graficar_senal_tiempo(t, y, 'Señal Moduladora AM en el Tiempo', 'Tiempo [s]', 'Amplitud')

        # Transformada de Fourier de la señal modulada
        st.subheader('Transformada de Fourier de la Señal Modulada')
        graficar_fft(t, y, 'Espectro de la Señal Modulada')

        # Reproducción de audio
        st.subheader('Reproducción de Audio')

        # Convertir el segmento de datos a WAV en un buffer de BytesIO
        wav_buffer = io.BytesIO()
        sf.write(wav_buffer, data_segment, sample_rate, format='wav')
        wav_buffer.seek(0)  # Reiniciar el puntero del buffer

        st.audio(wav_buffer, format='audio/wav')

        os.remove("audio.wav")

if __name__ == "__main__":
    main()

Overwriting Modulacion.py


In [ ]:
#cambiar su token
token = '' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [ ]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run Modulacion.py --server.port 8400 & #cambiar el puerto 8499 pueden poner cualquier número de 4 digitos

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='8400', proto='http', bind_tls=True) #acá tambien lo tienen que cambiar

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://930b-34-16-142-233.ngrok-free.app


In [ ]:
#exit("Stopping the execution")